In [ ]:
# Install necessary libraries
!pip install textblob tensorflow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from textblob import TextBlob
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Conv1D, MaxPooling1D, Dense, Dropout, Input, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive

def load_data(file_path):
    df = pd.read_csv(file_path, low_memory=False,nrows=1000)
    df.rename(columns={'comments_orig': 'comments'}, inplace=True)
    df['clean_title'] = df['clean_title'].astype(str).str.lower()
    df['comments'] = df['comments'].astype(str)
    return df

def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

def process_comments(row):
    try:
        comments = row['comments'].split('|__|')
    except AttributeError:
        return 0, 0
    polarities, subjectivities = zip(*[get_sentiment(str(comment)) for comment in comments]) if comments else ([0], [0])
    return np.mean(polarities), np.mean(subjectivities)

def preprocess_data(df):
    df['comments_polarity'], df['comments_subjectivity'] = zip(*df.apply(process_comments, axis=1))
    df['clean_title_polarity'], df['clean_title_subjectivity'] = zip(*df['clean_title'].apply(get_sentiment))
    df['separated_comment'] = df['comments'].apply(lambda x: x.split('|__|'))
    return df

def prepare_features_and_labels(df):
    X = df[['clean_title', 'score', 'separated_comment', 'subreddit', 'num_comments', 'upvote_ratio']]
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df['2_way_label'].astype(str))
    return train_test_split(X, y, test_size=0.2, random_state=42), label_encoder

def build_model(max_words=10000, max_len=100):
    text_input = Input(shape=(max_len,))
    embedding_layer = Embedding(input_dim=max_words, output_dim=128, input_length=max_len)(text_input)
    conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
    maxpool_layer = MaxPooling1D(pool_size=2)(conv_layer)
    bilstm_layer = Bidirectional(LSTM(128, return_sequences=False))(maxpool_layer)
    sentiment_input = Input(shape=(4,))
    merged = Concatenate()([bilstm_layer, sentiment_input])
    dense_layer = Dense(128, activation='relu')(merged)
    dropout_layer = Dropout(0.5)(dense_layer)
    output_layer = Dense(1, activation='sigmoid')(dropout_layer)
    model = Model(inputs=[text_input, sentiment_input], outputs=output_layer)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def evaluate_model(model, X_test_pad, X_test_sentiment, y_test, label_encoder):
    y_pred = (model.predict([X_test_pad, X_test_sentiment]) > 0.5).astype(int)
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

def main():
    drive.mount('/content/drive', force_remount=True)
    file_path = "/content/drive/My Drive/datasets/merged_cleaned_data_v34_no_photo.csv"
    df = load_data(file_path)
    df = preprocess_data(df)
    (X_train, X_test, y_train, y_test), label_encoder = prepare_features_and_labels(df)

    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(X_train['clean_title'])
    X_train_seq = tokenizer.texts_to_sequences(X_train['clean_title'])
    X_test_seq = tokenizer.texts_to_sequences(X_test['clean_title'])
    X_train_pad = pad_sequences(X_train_seq, maxlen=100)
    X_test_pad = pad_sequences(X_test_seq, maxlen=100)
    X_train_sentiment = df.loc[X_train.index, ['clean_title_polarity', 'clean_title_subjectivity', 'comments_polarity', 'comments_subjectivity']].values
    X_test_sentiment = df.loc[X_test.index, ['clean_title_polarity', 'clean_title_subjectivity', 'comments_polarity', 'comments_subjectivity']].values

    model = build_model()
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model.fit([X_train_pad, X_train_sentiment], y_train, epochs=10, batch_size=128, validation_data=([X_test_pad, X_test_sentiment], y_test), callbacks=[early_stopping])
    evaluate_model(model, X_test_pad, X_test_sentiment, y_test, label_encoder)

if __name__ == "__main__":
    main()


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 554ms/step - accuracy: 0.5156 - loss: 0.6876 - val_accuracy: 0.5200 - val_loss: 0.7093
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 453ms/step - accuracy: 0.5989 - loss: 0.6608 - val_accuracy: 0.5950 - val_loss: 0.6560
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 612ms/step - accuracy: 0.7190 - loss: 0.5742 - val_accuracy: 0.6350 - val_loss: 0.6818
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 455ms/step - accuracy: 0.7516 - loss: 0.4962 - val_accuracy: 0.6500 - val_loss: 0.6699
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 456ms/step - accuracy: 0.8514 - loss: 0.3625 - val_accuracy: 0.7050 - val_loss: 0.6061
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 605ms/step - accuracy: 0.9206 - loss: 0.2361 - val_accuracy: 0.6700 - val_loss: 0.8830
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 569ms/step - accuracy: 0.9735 - loss: 0.0966 - val_accuracy: 0.7200 - val_loss: 0.9974
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 456ms/step - accuracy: 0.9848 - loss: 0.0528 - val_accuracy: 0.7250 - val_loss